In [20]:
#importing all necessary libraries for the program
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D 
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L2
import time
from tensorflow.keras.callbacks import TensorBoard

In [21]:
# definig some variables
IMG_SIZE = 50

# note: the training and testing set 
#should be in this folder to run the program correctly

DATADIR = "C:/Assignment_images"  
test_dir = "C:/test_set"
CATEGORIES = ["cats", "dogs"]
NAME = f"cats-vs-dogs-cnn-64x2-{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{NAME}")

In [13]:
# creating a training data set
training_data = []
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) # path to cats or dogs directory
        class_num = CATEGORIES.index(category)
        
        for img in os.listdir(path):
            # using try and except to pass all the images that have problem in resizeing
            try: 
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                image_std = new_array.astype('float') / 255.0
                training_data.append([image_std, class_num])
            except Exception as e:
                pass
create_training_data()

In [14]:
# creating a testing data set
testing_data = []

def create_testing_data():
    for category in CATEGORIES:
        test_path = os.path.join(test_dir, category) # path to cats or dogs directory
        class_num = CATEGORIES.index(category)
        for img in os.listdir(test_path):
            # using try and except to pass all the images that have problem in resizeing
            try:
                img_array = cv2.imread(os.path.join(test_path,img))
                test_new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                image_std = test_new_array.astype('float') / 255.0
                testing_data.append([image_std, class_num])
            except Exception as e:
                pass
create_testing_data()

In [15]:
import random
random.shuffle(training_data)

In [16]:
train_features = []
train_label = []
test_features = []
test_label = []

In [17]:
# breaking trainig_data and testing_data into features and label and turning into numoy array.
train_features = np.array([i[0] for i in training_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
test_features = np.array([i[0] for i in testing_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
test_label = np.array([i[1] for i in testing_data])
train_label = np.array([i[1] for i in training_data])

In [18]:
from tensorflow.keras.optimizers import Adam

opt_1 = Adam(learning_rate=0.001)

In [19]:
# creating the model and adding hyperparameters to have better accuracy
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = train_features.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # this convert our 3D features map into 1D vectors

model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
             optimizer = opt_1,
             metrics = ["accuracy"])

model.fit(train_features, train_label, batch_size=32, epochs= 5, validation_data =(test_features, test_label), callbacks=[tensorboard])

Epoch 1/5
251/251 [==============================] - 43s 159ms/step - loss: 0.6815 - accuracy: 0.5631 - val_loss: 0.6672 - val_accuracy: 0.5885
Epoch 2/5
251/251 [==============================] - 34s 135ms/step - loss: 0.6005 - accuracy: 0.6802 - val_loss: 0.5660 - val_accuracy: 0.7190
Epoch 3/5
251/251 [==============================] - 33s 130ms/step - loss: 0.5354 - accuracy: 0.7334 - val_loss: 0.5618 - val_accuracy: 0.7125
Epoch 4/5
251/251 [==============================] - 29s 116ms/step - loss: 0.4798 - accuracy: 0.7707 - val_loss: 0.5067 - val_accuracy: 0.7550
Epoch 5/5
251/251 [==============================] - 35s 139ms/step - loss: 0.4329 - accuracy: 0.8017 - val_loss: 0.5006 - val_accuracy: 0.7705
